In [1]:
import uuid

from langchain_ollama import OllamaLLM
from langchain_core.runnables import RunnableSequence, RunnableBranch, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [2]:
llm = OllamaLLM(model="llama3:8b")

In [3]:
identification_template = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a classification assistant. "
        "Your task is to determine if the user's question is specifically about SQL-related tasks, queries, or commands. "
        "Respond only with 'True' if it is related to SQL, otherwise respond with 'False'. "
        "Return exactly one word: 'True' or 'False'."
        "Examples:\n"
        "Q: How do I get users below age 18? → True\n"
        "Q: What is Python? → False\n"
        "Q: List all orders placed in the last 7 days → True\n"
        "Q: Tell me a joke → False\n"
        "Only return one word: True or False."
    )),
    ("human", "Is the following question about SQL? Question: {question}")
])
identification_chain = RunnableSequence(identification_template, llm)

def identify(prompt: MessagesState):
    print(prompt[len(prompt) - 1].content)
    result = identification_chain.invoke({"question": prompt[len(prompt) - 1].content})
    return result.lower() == "true"

In [4]:
general_template = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a helpful and friendly AI assistant. "
        "You provide clear, concise, and accurate answers to user questions across a wide range of topics, including technology, programming, general knowledge, and day-to-day tasks. "
        "If you are uncertain or need more information, ask clarifying questions rather than guessing."
    )),
    ("human", "{question}")
])
general_chain = RunnableSequence(general_template, llm)

def general_chat(prompt):
    return general_chain.invoke({"question": prompt})

general_lambda = RunnableLambda(general_chat)

In [5]:
sql_template = ChatPromptTemplate.from_messages([
    ("system", (
        "You are an expert SQL query generator. "
        "Your job is to convert natural language questions into syntactically correct and production-ready SQL queries. "
        "Always return only the SQL code block. "
        "Do not provide explanations, comments, or anything else. "
        "If the user’s question is unclear or missing required information like table or column names, respond with a clarifying question instead of guessing."
    )),
    ("human", "Convert this question into SQL: {question}")
])
sql_chain = RunnableSequence(sql_template, llm)

def sql_chat(prompt):
    return sql_chain.invoke({"question": prompt})

sql_lambda = RunnableLambda(sql_chat)

In [6]:
def chat(state: MessagesState):
    branch = RunnableBranch(
        (identify, sql_lambda),
        general_lambda
    )
    return {"messages": branch.invoke(state["messages"])}

In [7]:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_node("llm", chat)
workflow.add_edge(START, "llm")

memory = MemorySaver()

chat_bot = workflow.compile(checkpointer=memory)

config = {
    "configurable": {
        "thread_id": uuid.uuid4()
    }
}

In [8]:
question = ""

while question != "/bye":
    question = str(input())
    for event in chat_bot.stream({"messages": [question]}, config, stream_mode="values"):
        event["messages"][-1].pretty_print()

 Hi


================================ Human Message =================================

Hi
Hi
================================ Human Message =================================

Hello! I'm thrilled to be assisting you. What's on your mind today? Do you have a specific question or topic you'd like to discuss?


 I would like to ask your to visualize my app data from db


================================ Human Message =================================

I would like to ask your to visualize my app data from db
I would like to ask your to visualize my app data from db
================================ Human Message =================================

SELECT * FROM db;


 U got it wrong


================================ Human Message =================================

U got it wrong
U got it wrong
================================ Human Message =================================

I'm happy to help! However, I need a bit more information from you. It seems like you'd like me to visualize your app data from the database, but there's a query missing. 

Could you please provide the SQL query that retrieves the data you're interested in visualizing? Additionally, what type of visualization are you looking for (e.g., chart, graph, table)?


 find all users name and age = 19 whose name is navin


================================ Human Message =================================

find all users name and age = 19 whose name is navin
find all users name and age = 19 whose name is navin
================================ Human Message =================================

SELECT u.name, u.age
FROM users u
WHERE u.name = 'navin' AND u.age = 19;


 /bye


================================ Human Message =================================

/bye
/bye
================================ Human Message =================================

I'd be happy to help you with that! However, I need a bit more information from you. It seems like you'd like me to visualize your app data from the database, but there's a query missing.

Could you please provide the SQL query that retrieves the data you're interested in visualizing? Additionally, what type of visualization are you looking for (e.g., chart, graph, table)?

Also, I noticed that you mentioned finding all users with name "navin" and age 19. It seems like you've provided a SQL query to achieve this. Is there anything specific you'd like me to do with these results?
